# Simple FEM for an elastic plate with a hole
Should be replaced by an IGA variant

## Create a mesh for the plate with hole

In [12]:
import numpy
import treelog

from nutils import export, function, mesh, solver, testing
from nutils.expression_v2 import Namespace

from global_constants import EPS0, LBD, MU, B0, X0, Y0, L, R

# Element size
N = 30
s = L / N
etype = "square"
btype = "std"
degree = 2
poisson = LBD / (2*(LBD + MU))
traction = 0.1

# Build geometry
domain0, geom = mesh.unitsquare(N, etype)
domain = domain0.trim(function.norm2(geom) - R, maxrefine=2)

## ...

In [13]:
ns = Namespace()
ns.δ = function.eye(domain.ndims)
ns.x = geom
ns.define_for("x", gradient="∇", normal="n", jacobians=("dV", "dS"))
ns.lmbda = LBD
ns.mu = MU
ns.ubasis = domain.basis(btype, degree=degree).vector(2)
ns.u = function.dotarg("lhs", ns.ubasis)
ns.X_i = "x_i + u_i"
ns.strain_ij = "(∇_j(u_i) + ∇_i(u_j)) / 2"
ns.stress_ij = "lmbda strain_kk δ_ij + 2 mu strain_ij"
ns.r2 = "x_k x_k"
ns.R2 = R**2 / ns.r2
ns.k = (3 - poisson) / (1 + poisson)  # plane stress parameter
ns.scale = traction * (1 + poisson) / 2
ns.uexact_i = "scale (x_i ((k + 1) (0.5 + R2) + (1 - R2) R2 (x_0^2 - 3 x_1^2) / r2) - 2 δ_i1 x_1 (1 + (k - 1 + R2) R2))"
ns.du_i = "u_i - uexact_i"

In [14]:
sqr = domain.boundary["left,bottom"].integral(
    "(u_i n_i)^2 dS" @ ns, degree=degree * 2
)
cons = solver.optimize("lhs", sqr, droptol=1e-15)
sqr = domain.boundary["top,right"].integral("du_k du_k dS" @ ns, degree=20)
cons = solver.optimize("lhs", sqr, droptol=1e-15, constrain=cons)

res = domain.integral("∇_j(ubasis_ni) stress_ij dV" @ ns, degree=degree * 2)
lhs = solver.solve_linear("lhs", res, constrain=cons)

bezier = domain.sample("bezier", 5)
X, stressxx = bezier.eval(["X_i", "stress_00"] @ ns, lhs=lhs)
export.triplot("stressxx.png", X, stressxx, tri=bezier.tri, hull=bezier.hull)

err = (
    domain.integral(
        function.stack(["du_k du_k dV", "∇_j(du_i) ∇_j(du_i) dV"] @ ns),
        degree=max(degree, 3) * 2,
    ).eval(lhs=lhs)
    ** 0.5
)
treelog.user("errors: L2={:.2e}, H1={:.2e}".format(*err))

optimize > constrained 94/7146 dofs
optimize > optimum value 0.00e+00
optimize > solve > solving 240 dof system to machine precision using arnoldi solver
optimize > solve > solver returned with residual 1e-17
optimize > constrained 334/7146 dofs
optimize > optimum value 1.11e-15
solve > solving 6812 dof system to machine precision using arnoldi solver
solve > solver returned with residual 3e-15
stressxx.png
errors: L2=2.44e-03, H1=1.11e-02


## Postprocessing

In [15]:
# Compute stress